In [1]:
import os
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
tqdm.pandas() 



'def list_csv_files(directory):\n    csv_files = []\n    for filename in os.listdir(directory):\n        if filename.endswith(".csv"):\n            csv_files.append(filename)\n    return csv_files\n'

In [3]:
geo_data = ['latitude_degree','longitude_degree']

independent_signals = ['accelerator_pedal','accelerator_pedal_gradient_sign','brake_pressure','steering_angle_calculated','steering_angle_calculated_sign']
dependent_signals = ['vehicle_speed','roll_angle','pitch_angle']


Spliting Data

In [6]:
class DataPreprocessing():
    def __init__(self, directory, train_size=.9, do_train_test_split = True):
        
      self.directory = directory
      self.do_train_test_split = do_train_test_split
      self.list_of_path = DataPreprocessing.list_csv_files(directory)
      self.train_size = train_size
      self.train_df_batch = []
      self.test_df_batch = []
      self.dataset_batch = []
      self.train_sequences = []
      self.test_sequences = []
      self.scaler = MinMaxScaler(feature_range=(-1,1))
    
    @staticmethod
    def split_dataset(data, train_size):
      train_size = int(len(data)* train_size)
      train_df, test_df = data[:train_size], data[train_size + 1:]
      return train_df, test_df  

    @staticmethod
    def list_csv_files(directory):
      csv_files = []
      for filename in os.listdir(directory):
        if filename.endswith(".csv"):
          csv_files.append(filename)
      return csv_files    
    
    def scaling_fit(self):
      self.scaler.fit(pd.concat(self.train_df_batch))

    def scaling_transform(self, dataset):
      return pd.DataFrame(self.scaler.transform(dataset),
                          index=dataset.index,
                          columns=dataset.columns)
    
    def data_loader(self, file_path):

      path = '{}{}'.format(self.directory, file_path)
      print(path)
      dataset = pd.read_csv(path)
      dataset.timestamp = pd.to_datetime(dataset.timestamp)
      dataset = dataset.set_index('timestamp')
      return dataset

    def fit(self): 
      for each_file_path in self.list_of_path:
        # Loading data from each file
        dataset = self.data_loader(each_file_path)

        # Calculate min and max value of all features this will be used for scaling
        self.dataset_batch.append(dataset)
        if self.do_train_test_split:
          train_df, test_df = DataPreprocessing.split_dataset(dataset, self.train_size)
          self.train_df_batch.append(train_df)
          self.test_df_batch.append(test_df)

      # Scaling
      self.scaling_fit()

      # Scaling transform
      for batch_idx in range(len(self.train_df_batch)):
        self.train_df_batch[batch_idx] = (self.scaling_transform(self.train_df_batch[batch_idx]))
        self.test_df_batch[batch_idx] = self.scaling_transform(self.test_df_batch[batch_idx])

    def create_sequences_for_a_batch(input_data, target_colums, sequence_length): 

      sequences = []
      data_size = len(input_data)

      for i in range(data_size - sequence_length):
        sequence = input_data[i: i+sequence_length]

        label = input_data[i: i + sequence_length][target_colums]

        sequences.append((sequence, label))

  



    # Getter method
    @property
    def get_dataset(self):
      return self.dataset   

    @property
    def get_train_df(self):
      return self.train_df_batch

    @property
    def get_test_df(self):
      return self.test_df_batch 
    

      
     # TODO create sequence method
      
      

In [7]:
obj1 = DataPreprocessing('datasets/')

In [9]:
obj1.fit()

datasets/resampled_20180810150607.csv
datasets/resampled_20190401121727.csv
datasets/resampled_20190401145936.csv
